# Hand Tracking Model

## Load imports

In [1]:
# Przetwarzanie danych
import pandas as pd
import numpy as np

# Skalowanie i podział danych
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Ewaluacja
from sklearn.metrics import accuracy_score, classification_report

# Model
from xgboost import XGBClassifier

# Zapis modelu i skalera
import pickle


## Load dataset

In [2]:
df = pd.read_csv("gestures_dataset.csv")
print("✅ Dane wczytane.")
df.head()

✅ Dane wczytane.


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,label
0,0.494490,0.708071,2.878070e-07,0.524123,0.677799,-0.032304,0.546674,0.630880,-0.047820,0.565645,...,0.454037,0.495310,-0.025063,0.442520,0.460195,-0.032691,0.434146,0.425948,-0.037496,OPEN
1,0.494238,0.707083,3.094904e-07,0.523785,0.676861,-0.033344,0.546862,0.631029,-0.049547,0.566174,...,0.453692,0.495877,-0.023847,0.442023,0.461033,-0.031497,0.433738,0.427381,-0.036450,OPEN
2,0.525618,0.723417,2.485319e-07,0.561070,0.677013,-0.016650,0.582363,0.608424,-0.024285,0.600644,...,0.450767,0.524465,-0.037607,0.438585,0.490851,-0.044003,0.429751,0.455712,-0.047923,OPEN
3,0.443406,0.533408,3.308509e-07,0.492958,0.479014,-0.023147,0.524925,0.393028,-0.032627,0.547941,...,0.355848,0.259022,-0.050706,0.339588,0.204934,-0.061433,0.326375,0.152666,-0.068224,OPEN
4,0.431041,0.538612,3.439229e-07,0.478237,0.484201,-0.023533,0.509063,0.395912,-0.033089,0.532328,...,0.340244,0.261701,-0.048480,0.323630,0.207409,-0.058714,0.310164,0.153941,-0.065264,OPEN


## Split dataset into features and labels

In [3]:
X = df.drop(columns=["label"])
y = df["label"]

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Unique labels:", y.unique())

X shape: (1705, 63)
y shape: (1705,)
Unique labels: ['OPEN' 'CLOSE' 'POINTER']


## Data normalization

In [4]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("✅ Dane znormalizowane.")

✅ Dane znormalizowane.


## Encoder

In [5]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("✅ Etykiety zakodowane.")
print("Oryginalne pierwsze 5:", y.values[:5])
print("Zakodowane pierwsze 5:", y_encoded[:5])
print("Mapowanie (Klasy):")
for i, class_name in enumerate(label_encoder.classes_):
    print(f"{i}: {class_name}")

✅ Etykiety zakodowane.
Oryginalne pierwsze 5: ['OPEN' 'OPEN' 'OPEN' 'OPEN' 'OPEN']
Zakodowane pierwsze 5: [1 1 1 1 1]
Mapowanie (Klasy):
0: CLOSE
1: OPEN
2: POINTER


## Train and test split data

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print("✅ Podzielono dane.")
print("Trening X:", X_train.shape, "| Test X:", X_test.shape) # ADDED Line
print("Trening y:", y_train.shape, "| Test y:", y_test.shape) # ADDED Line
print("Typ y_train:", y_train.dtype)

✅ Podzielono dane.
Trening X: (1364, 63) | Test X: (341, 63)
Trening y: (1364,) | Test y: (341,)
Typ y_train: int64


## Model training

In [7]:
model = XGBClassifier(
    n_estimators=150,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.9,
    objective='multi:softmax',
    num_class=len(label_encoder.classes_),
    use_label_encoder=False,
    eval_metric="mlogloss"
)

model.fit(X_train, y_train)
print("✅ Model wytrenowany.")


/Volumes/bartekdanko/Python/SpotifyWCAG/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:37:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Model wytrenowany.


In [8]:
y_pred = model.predict(X_test)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification report:\n")
print(classification_report(y_test, y_pred))

✅ Accuracy: 0.9912023460410557

📊 Classification report:

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        94
           1       0.99      1.00      1.00       121
           2       1.00      0.98      0.99       126

    accuracy                           0.99       341
   macro avg       0.99      0.99      0.99       341
weighted avg       0.99      0.99      0.99       341



In [9]:
# Model
with open("gesture_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Skaler
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

# LabelEncoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("✅ Zapisano: gesture_model.pkl, scaler.pkl, label_encoder.pkl")

✅ Zapisano: gesture_model.pkl, scaler.pkl, label_encoder.pkl
